In [12]:
!unzip spam_ham_dataset.csv.zip

Archive:  spam_ham_dataset.csv.zip
replace spam_ham_dataset.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: spam_ham_dataset.csv    


In [50]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from sklearn.model_selection import train_test_split

In [14]:
data = pd.read_csv("spam_ham_dataset.csv")

In [15]:
data

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5166,1518,ham,Subject: put the 10 on the ft\r\nthe transport...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms\r\nhp...,0
5168,2933,ham,Subject: calpine daily gas nomination\r\n>\r\n...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0


In [16]:
spam_mail = data.loc[data.label_num == 1]
valid_mail = data.loc[data.label_num == 0].head(spam_mail.shape[0])

In [17]:
mails = pd.concat([spam_mail,valid_mail],axis=0)

In [18]:
mails = mails.sort_index()

In [19]:
mails

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0
...,...,...,...,...
5159,4381,spam,Subject: pictures\r\nstreamlined denizen ajar ...,1
5161,4979,spam,Subject: penny stocks are about timing\r\nnoma...,1
5162,4162,spam,Subject: anomaly boys from 3881\r\nuosda apapr...,1
5164,4365,spam,Subject: slutty milf wants to meet you\r\ntake...,1


In [28]:
encoder = hub.KerasLayer("https://www.kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/bert-en-uncased-l-10-h-128-a-2/versions/2")
preprocess = hub.KerasLayer("https://kaggle.com/models/tensorflow/bert/frameworks/TensorFlow2/variations/en-uncased-preprocess/versions/3")

In [51]:
x_train,x_val,y_train,y_val = train_test_split(mails.text,mails.label_num,random_state=0)

In [58]:
# setting bert layers
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string,name="Input")
preprocessed_text = preprocess(text_input)
output = encoder(preprocessed_text)

# setting model layers
l1 = tf.keras.layers.Dropout(0.1,name="Dropout")(output["pooled_output"])
l2 = tf.keras.layers.Dense(1,name="Danse")(l1)

model = tf.keras.Model(inputs=[text_input],outputs=[l2])


In [59]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Input (InputLayer)          [(None,)]                    0         []                            
                                                                                                  
 keras_layer_1 (KerasLayer)  {'input_mask': (None, 128)   0         ['Input[0][0]']               
                             , 'input_type_ids': (None,                                           
                              128),                                                               
                              'input_word_ids': (None,                                            
                             128)}                                                                
                                                                                              

In [60]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [61]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
71/71 [==============================] - 50s 575ms/step - loss: 2.3828 - accuracy: 0.5534 - precision: 0.5579 - recall: 0.5559
Epoch 2/10
71/71 [==============================] - 42s 589ms/step - loss: 2.3113 - accuracy: 0.5658 - precision: 0.6305 - recall: 0.3383
Epoch 3/10
71/71 [==============================] - 41s 580ms/step - loss: 1.9855 - accuracy: 0.5649 - precision: 0.5556 - recall: 0.6907
Epoch 4/10
71/71 [==============================] - 45s 628ms/step - loss: 2.2423 - accuracy: 0.5721 - precision: 0.5638 - recall: 0.6731
Epoch 5/10
71/71 [==============================] - 44s 625ms/step - loss: 6.0543 - accuracy: 0.5089 - precision: 0.5070 - recall: 0.9833
Epoch 6/10
71/71 [==============================] - 43s 612ms/step - loss: 4.1957 - accuracy: 0.5418 - precision: 0.5256 - recall: 0.9507
Epoch 7/10
71/71 [==============================] - 43s 608ms/step - loss: 2.4883 - accuracy: 0.5587 - precision: 0.5419 - recall: 0.8141
Epoch 8/10
71/71 [================

In [62]:
model.evaluate(x_val,y_val)

24/24 [==============================] - 15s 567ms/step - loss: 0.9242 - accuracy: 0.6627 - precision: 0.6628 - recall: 0.6209


[0.9241509437561035, 0.6626666784286499, 0.662756621837616, 0.6208791136741638]